In [16]:
try:
    import vectorbt
except ModuleNotFoundError as e:
    !pip install vectorbt    

try:
    import yfinance
except ModuleNotFoundError as e:
    !pip install yfinance   

try:
    import ccxt
except ModuleNotFoundError as e:
    !pip install ccxt

try:
    import talib
except ModuleNotFoundError as e:
    !pip install talib_binary         


from IPython.display import clear_output
clear_output()

!exit

In [17]:

import vectorbt as vbt
import talib
import pandas as pd


In [18]:
#@title 回測設定
symbol_name = "SOL/USDT" #@param {type:"string"}

#timeframe = ['1m', '5m', '15m', '1h', '4h', '1d', '3d']
timeframe = ['1h','4h','1d']

date_start = "2021-01-01" #@param {type:"date"}
date_end = "2022-01-01" #@param {type:"date"}

ema_s_start = 12 #@param {type:"integer"}
ema_s_end = 14 #@param {type:"integer"}

ema_m_start = 50 #@param {type:"integer"}
ema_m_end = 53 #@param {type:"integer"}

sma_deduct_start = 20 #@param {type:"integer"}
sma_deduct_end = 23 #@param {type:"integer"}


init_cash = 100 #@param {type:"integer"}



In [19]:
import warnings
warnings.filterwarnings('ignore')

result = pd.DataFrame()
for i_timeframe in timeframe:
    data = vbt.CCXTData.download(
        symbols=symbol_name,
        start=date_start,
        end=date_end,
        timeframe=i_timeframe)

    ohlc_df = data.get("Close")
    
    for i_ema_s in range(ema_s_start , ema_s_end):
        #print(f"emasss:  {i_ema_s}")

        for i_ema_m in range(ema_m_start , ema_m_end):
            #print(f"emammm:  {i_ema_m}")
            ema_sline = talib.EMA(ohlc_df,timeperiod=i_ema_s)
            ema_mline = talib.EMA(ohlc_df,timeperiod=i_ema_m)
                
            ema_entry = [False]
            ema_exit = [False]
                
            for i in range(1,ema_sline.shape[0]):
                if ema_sline[i] > ema_mline[i] and ema_sline[i-1] < ema_mline[i-1]:
                    ema_entry.append(True)
                else:
                    ema_entry.append(False)

                if ema_sline[i] < ema_mline[i] and ema_sline[i-1] > ema_mline[i-1]:
                    ema_exit.append(True)
                else:
                    ema_exit.append(False)

            ema_pf = vbt.Portfolio.from_signals(
                    close=ohlc_df, 
                    entries=ema_entry,
                    exits=ema_exit, 
                    init_cash=init_cash)

            ema_sr = pd.Series([i_timeframe,i_ema_s,i_ema_m,0],index=["timeframe","ema_s","ems_m","sma_deduct"]) 
            ema_portfolio = pd.concat([ema_sr,ema_pf.stats()])
            result = pd.concat([result,ema_portfolio],axis=1,ignore_index=True)   
            
                        
            for i_sma_d in range(sma_deduct_start,sma_deduct_end):
                sma_d_exit = [False]
                for i in range(1,ohlc_df.shape[0]):
                    if ohlc_df[i] < ohlc_df[i-i_sma_d]:
                        sma_d_exit.append(True)
                    else:
                        sma_d_exit.append(False)    
                
                sma_pf = vbt.Portfolio.from_signals(
                    close=ohlc_df, 
                    entries=ema_entry,
                    exits=sma_d_exit, 
                    init_cash=init_cash)       

                sma_sr = pd.Series([i_timeframe,i_ema_s,i_ema_m,i_sma_d],index=["timeframe","ema_s","ems_m","sma_deduct"])
                sma_portfolio = pd.concat([sma_sr,sma_pf.stats()])
                result = pd.concat([result,sma_portfolio],axis=1,ignore_index=True)   
           

clear_output() 
result = result.T
print(f"backtimes: {result.shape[0]}")   

backtimes: 72


In [20]:
result

,timeframe,ema_s,ems_m,sma_deduct,Start,End,Period,Start Value,End Value,Total Return [%],...,Avg Winning Trade [%],Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio
0,1h,12,50,0,2020-12-31 16:00:00+00:00,2021-12-31 15:00:00+00:00,8747,100.0,2304.566555,2204.566555,...,16.548,-3.441663,97.405405,23.083333,2.108658,22.72749,NaN,NaN,NaN,NaN
1,1h,12,50,20,2020-12-31 16:00:00+00:00,2021-12-31 15:00:00+00:00,8747,100.0,291.750376,191.750376,...,7.04025,-3.0008,30.078947,13.173913,1.530566,2.282743,NaN,NaN,NaN,NaN
2,1h,12,50,21,2020-12-31 16:00:00+00:00,2021-12-31 15:00:00+00:00,8747,100.0,256.932625,156.932625,...,7.151444,-2.892986,32.485714,13.673913,1.531098,1.93744,NaN,NaN,NaN,NaN
3,1h,12,50,22,2020-12-31 16:00:00+00:00,2021-12-31 15:00:00+00:00,8747,100.0,371.808377,271.808377,...,8.704843,-2.8819,35.764706,14.869565,1.631964,3.397605,NaN,NaN,NaN,NaN
4,1h,12,51,0,2020-12-31 16:00:00+00:00,2021-12-31 15:00:00+00:00,8747,100.0,2649.838429,2549.838429,...,17.201775,-3.42745,98.916667,24.706897,2.226445,27.125941,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,1d,13,51,22,2021-01-01 00:00:00+00:00,2021-12-31 00:00:00+00:00,365 days 00:00:00,100.0,380.480984,280.480984,...,280.480984,NaN,52 days 00:00:00,NaT,inf,280.480984,2.451576,8.028994,2.602138,6.141967
68,1d,13,52,0,2021-01-01 00:00:00+00:00,2021-12-31 00:00:00+00:00,365 days 00:00:00,100.0,339.786649,239.786649,...,406.459732,-32.909444,127 days 00:00:00,16 days 00:00:00,8.286256,119.893325,1.875186,6.481073,1.542353,3.259767
69,1d,13,52,20,2021-01-01 00:00:00+00:00,2021-12-31 00:00:00+00:00,365 days 00:00:00,100.0,380.480984,280.480984,...,280.480984,NaN,52 days 00:00:00,NaT,inf,280.480984,2.451576,8.028994,2.602138,6.141967
70,1d,13,52,21,2021-01-01 00:00:00+00:00,2021-12-31 00:00:00+00:00,365 days 00:00:00,100.0,389.065996,289.065996,...,289.065996,NaN,51 days 00:00:00,NaT,inf,289.065996,2.489424,8.274747,2.664221,6.256314


In [21]:
#依獲利排序，取前100, 存到csv檔
result.sort_values("End Value", ascending=False, inplace=True)
result.head(100).to_csv(f"{symbol_name.replace('/','')}_endvaluetable.csv")
result.head(100)

,timeframe,ema_s,ems_m,sma_deduct,Start,End,Period,Start Value,End Value,Total Return [%],...,Avg Winning Trade [%],Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio
40,4h,13,51,0,2020-12-31 16:00:00+00:00,2021-12-31 12:00:00+00:00,365 days 00:00:00,100.0,2747.568528,2647.568528,...,122.910419,-6.902853,25 days 11:25:42.857142857,4 days 04:20:00,3.879489,139.345712,3.198446,48.797804,1.279073,5.313723
16,1h,13,51,0,2020-12-31 16:00:00+00:00,2021-12-31 15:00:00+00:00,8747,100.0,2702.402936,2602.402936,...,17.165876,-3.521317,101.083333,24.545455,2.31007,28.597834,NaN,NaN,NaN,NaN
20,1h,13,52,0,2020-12-31 16:00:00+00:00,2021-12-31 15:00:00+00:00,8747,100.0,2695.292599,2595.292599,...,17.544813,-3.657954,104.057143,26.115385,2.262485,29.830949,NaN,NaN,NaN,NaN
32,4h,12,52,0,2020-12-31 16:00:00+00:00,2021-12-31 12:00:00+00:00,365 days 00:00:00,100.0,2656.368812,2556.368812,...,108.248224,-7.327973,23 days 09:30:00,3 days 10:40:00,3.377265,127.818441,3.172379,47.657933,1.276503,5.268698
4,1h,12,51,0,2020-12-31 16:00:00+00:00,2021-12-31 15:00:00+00:00,8747,100.0,2649.838429,2549.838429,...,17.201775,-3.42745,98.916667,24.706897,2.226445,27.125941,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,4h,13,52,21,2020-12-31 16:00:00+00:00,2021-12-31 12:00:00+00:00,365 days 00:00:00,100.0,106.401678,6.401678,...,6.506567,-3.233323,4 days 06:51:25.714285714,1 days 20:21:49.090909090,1.151078,0.355649,0.392779,0.183028,1.065577,0.592393
34,4h,12,52,21,2020-12-31 16:00:00+00:00,2021-12-31 12:00:00+00:00,365 days 00:00:00,100.0,103.919804,3.919804,...,6.623823,-3.499867,4 days 00:00:00,2 days 02:10:54.545454545,1.084644,0.217767,0.352165,0.104966,1.058106,0.530067
38,4h,13,50,21,2020-12-31 16:00:00+00:00,2021-12-31 12:00:00+00:00,365 days 00:00:00,100.0,99.074204,-0.925796,...,6.732116,-3.988642,4 days 03:25:42.857142857,1 days 22:32:43.636363636,0.981012,-0.051433,0.267197,-0.025573,1.044185,0.400276
30,4h,12,51,21,2020-12-31 16:00:00+00:00,2021-12-31 12:00:00+00:00,365 days 00:00:00,100.0,98.856079,-1.143921,...,7.600152,-3.546438,4 days 22:00:00,1 days 19:40:00,0.975973,-0.063551,0.266257,-0.031706,1.043487,0.401044


In [22]:
#依獲利排序，取前100, 存到csv檔
result.sort_values("Win Rate [%]", ascending=False, inplace=True)
result.head(100).to_csv(f"{symbol_name.replace('/','')}_winratetable.csv")
result.head(100)

,timeframe,ema_s,ems_m,sma_deduct,Start,End,Period,Start Value,End Value,Total Return [%],...,Avg Winning Trade [%],Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio
71,1d,13,52,22,2021-01-01 00:00:00+00:00,2021-12-31 00:00:00+00:00,365 days 00:00:00,100.0,380.480984,280.480984,...,280.480984,NaN,52 days 00:00:00,NaT,inf,280.480984,2.451576,8.028994,2.602138,6.141967
66,1d,13,51,21,2021-01-01 00:00:00+00:00,2021-12-31 00:00:00+00:00,365 days 00:00:00,100.0,389.065996,289.065996,...,289.065996,NaN,51 days 00:00:00,NaT,inf,289.065996,2.489424,8.274747,2.664221,6.256314
59,1d,12,52,22,2021-01-01 00:00:00+00:00,2021-12-31 00:00:00+00:00,365 days 00:00:00,100.0,398.944436,298.944436,...,298.944436,NaN,53 days 00:00:00,NaT,inf,298.944436,2.523576,8.557525,2.653384,6.338428
53,1d,12,51,20,2021-01-01 00:00:00+00:00,2021-12-31 00:00:00+00:00,365 days 00:00:00,100.0,398.944436,298.944436,...,298.944436,NaN,53 days 00:00:00,NaT,inf,298.944436,2.523576,8.557525,2.653384,6.338428
55,1d,12,51,22,2021-01-01 00:00:00+00:00,2021-12-31 00:00:00+00:00,365 days 00:00:00,100.0,398.944436,298.944436,...,298.944436,NaN,53 days 00:00:00,NaT,inf,298.944436,2.523576,8.557525,2.653384,6.338428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,4h,12,51,21,2020-12-31 16:00:00+00:00,2021-12-31 12:00:00+00:00,365 days 00:00:00,100.0,98.856079,-1.143921,...,7.600152,-3.546438,4 days 22:00:00,1 days 19:40:00,0.975973,-0.063551,0.266257,-0.031706,1.043487,0.401044
28,4h,12,51,0,2020-12-31 16:00:00+00:00,2021-12-31 12:00:00+00:00,365 days 00:00:00,100.0,2084.440944,1984.440944,...,118.954574,-6.766589,24 days 12:00:00,3 days 18:08:00,3.169803,90.201861,2.999555,35.025585,1.25944,4.943255
24,4h,12,50,0,2020-12-31 16:00:00+00:00,2021-12-31 12:00:00+00:00,365 days 00:00:00,100.0,2104.217994,2004.217994,...,122.676344,-7.244436,24 days 09:08:34.285714285,3 days 18:08:00,3.14823,91.100818,3.01074,36.268075,1.261358,4.962182
35,4h,12,52,22,2020-12-31 16:00:00+00:00,2021-12-31 12:00:00+00:00,365 days 00:00:00,100.0,139.284689,39.284689,...,15.861682,-3.203188,6 days 08:00:00,2 days 08:00:00,1.734444,2.455293,0.839011,1.258661,1.136954,1.352018
